In [ ]:
!pip install finnhub-python
!pip install discord

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.8 MB/s eta 0:00:00


# IMPORTS

In [ ]:
import finnhub
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd 
#from google.colab import files
from google.colab import drive
import csv 
import json
import math
from discord import Webhook#, RequestsWebhookAdapter
import sched, time
#import zipfile

# DATA COLLECTION FUNCTIONS

In [ ]:
### Scrapes the current UNIX code
### DEPRECATED
'''
ERROR:
  When this is run repeatedly, the source website will eventually flag it. 
  Find a way to get the current unix without scraping. 
'''
def get_todays_unix():
  url = 'https://www.unixtimestamp.com/index.php'
  r = requests.get(url)
  soup = BeautifulSoup(r.content, 'html.parser')
  content = soup.find('div', class_='value epoch')
  content = re.sub('\s+', '', content.text)
  return int(content)

In [ ]:
def addSMA(close,length_of_comparison):
  result = []
  prices = close[::-1]
  for x in range(len(prices)):
    sma = 0
    if x + length_of_comparison <= len(close):
      for u in prices[x:x+length_of_comparison]:
        sma += u 
      sma /= length_of_comparison 
      result.append(sma)
    else:
      result.append(0)
  return result[::-1]

In [ ]:
### ORIGINAL FORMULA
def addRSI(close,length_of_comparison):
  result = []
  prices = close[::-1]
  for x in range(len(prices)):
    if x + length_of_comparison + 1 <= len(prices):
      up = 1 
      down = 1 
      for u in range(x,x+length_of_comparison):
        change = prices[u] - prices[u+1] 
        a = 1 / length_of_comparison
        if change > 0:
          avU = (a * change) + (1-a)
          up *= avU 
        if change < 0:
          avD = (a * abs(change)) + (1-a)
          down *= avD
      #up /= length_of_comparison 
      #down /= length_of_comparison
      rs = up/down
      rsi = 100 - (100/(1+rs))
      result.append(rsi)
    else:
      result.append(0)
  return result[::-1]

In [ ]:
### Adds SMA 20 - 200 Crosses
def add_crosses(sma20,sma200):
  result = []
  for x in range(len(sma20)):
    if sma20[x] > sma200[x]:
      result.append('golden cross')
    elif sma20[x] < sma200[x]:
      result.append('death cross')
    if sma20[x] == sma200[x]:
      result.append('')
  return result

In [ ]:
### Adds highs and lows of specified time period 
### Input data in reverse order 
def addPriceRange(highs,lows,timeperiod):
  result = []
  high_list = []
  low_list = []
  period = timeperiod
  for x in range(len(highs)):
    high = highs[x]
    low = lows[x]
    for y in highs[x:x+period]:
      if y > high:
        high = y 
    high_list.append(high)
    for u in lows[x:x+period]:
      if u < low:
        low = u
    low_list.append(low)
  result.append(high_list[::-1])
  result.append(low_list[::-1])
  return result
    

In [ ]:
### Find the price range score --> Returns a percentage value of how far the along the price range the current price is on the excel sheet
#### (0-1 Scale--> 0= low, 1=high)
def addPriceRangeScore(highs,lows,close):
  result = []
  for x in range(len(highs)):
    range52 = highs[x] - lows[x]
    numerator = close[x] - lows[x]
    if range52 > 0:
      score = numerator / range52
    else:
      score = 100
    result.append(score)
  return result

In [ ]:
### Adds a ratio of the difference between todays high and low prices and the high and low difference average of a specified time frame
def addMarketSpreadRatio(highs,lows,length_of_comparison):
  final = []
  high10 = highs[::-1]
  low10 = lows[::-1]
  for x in range(len(high10)):
    range10 = high10[x] - low10[x]
    av_range = 0
    if x+length_of_comparison <= len(high10):
      for u in range(x,x+length_of_comparison):
        next_range = high10[u] - low10[u]
        av_range += next_range 
      av_range /= length_of_comparison
      result = range10 / av_range 
      final.append(result)
    else:
      final.append(0)
  return final[::-1]

In [ ]:
### Adds relative volume over a specified time frame
def addRelVol(volume,length_of_comparison):
  vol = volume[::-1]
  result = []
  for x in range(len(vol)):
    currentVol = vol[x]
    avVol = 0
    if x+length_of_comparison <= len(vol):
      for u in range(x,x+length_of_comparison):
        avVol += vol[u]
      avVol /= length_of_comparison 
      relVol = currentVol / avVol 
      result.append(relVol)
    else:
      result.append(0)
  return result[::-1]

# COMPILE POLYGON.IO DATA 

In [ ]:
# returns data in reverse
def getPolygonData(tickers,start,end,timespan):
  ### start and end should be dates formatted as such: 2020-06-23
  apiKey = '<Key Here>'
  result = []
  t = []
  o = []
  h = []
  l = []
  c = []
  v = []
  sma20 = []
  sma200 = []
  for name in tickers:
    url = f'https://api.polygon.io/v2/aggs/ticker/{name}/range/1/{timespan}/{start}/{end}?apiKey={apiKey}'
    form = requests.get(url)
    data = form.content
    dataDict = json.loads(data).get('results')
    for entry in dataDict:
      t.append(entry.get('t'))
      o.append(entry.get('o'))
      h.append(entry.get('h'))
      l.append(entry.get('l'))
      c.append(entry.get('c'))
      v.append(entry.get('v'))
    rsi = addRSI(c,20)
    av20 = addSMA(c,20)
    av200 = addSMA(c,200)
    cross = add_crosses(av20,av200)
    high52 = addPriceRange(h[::-1],l[::-1],52)[0]
    low52 = addPriceRange(h[::-1],l[::-1],52)[1]
    rangescore = addPriceRangeScore(h,l,c)
    ###not taking EMA seriously in this data
    ema20 = av20 
    ema200 = av200
    ###
    high10 = addPriceRange(h[::-1],l[::-1],10)[0]
    low10 = addPriceRange(h[::-1],l[::-1],10)[1]
    spreadratio = addMarketSpreadRatio(high10,low10,52)
    relvol = addRelVol(v,52)
    final = {'t':t,'o':o,'h':h,'l':l,'c':c,'v':v,'rsi':rsi,'sma20':av20,'sma200':av200,
             'cross':cross, '52wkHigh':high52, '52wkLow':low52, 'rangescore':rangescore,'ema20':ema20,
             'ema200':ema200, '10dayHigh':high10, '10dayLow':low10, 'spreadRatio':spreadratio, 'relVol':relvol}
    stock = [final,name]
    result.append(stock)
  return result

# COMPILE FINNHUB DATA 

In [ ]:
### Scrapes financial data
#Returns List containing list of dictionaries 
# returns data in reverse
def getFinnhubData(user):
  #ARG user: list of tickers
  key= '<Key Here>'
  client = finnhub.Client(api_key=key)
  tickers = user
  result = []
  secperday = 86400
  today = int(time.time())
  av300 = int(today - 25920000)
  for i in tickers:
    stock = []
    data = client.technical_indicator(i,'D',av300, today, indicator='rsi', indicator_fields={"timeperiod":14})
    sma20 = client.technical_indicator(i,'D',av300, today, indicator='sma', indicator_fields={"timeperiod":20})
    sma200 = client.technical_indicator(i,'D',av300, today, indicator='sma', indicator_fields={"timeperiod":200})
    ema20 = client.technical_indicator(i,'D',av300, today, indicator='ema', indicator_fields={"timeperiod":20})
    ema200 = client.technical_indicator(i,'D',av300, today, indicator='ema', indicator_fields={"timeperiod":200})
    adx = client.technical_indicator(i,'D', av300, today, indicator='adx', indicator_fields={"timeperiod":20})
    data['sma20'] = sma20.get('sma')
    data['sma200'] = sma200.get('sma')
    data['cross'] = add_crosses(data['sma20'],data['sma200'])
    data['52wkHigh'] = addPriceRange(data['h'][::-1],data['l'][::-1],52)[0]
    data['52wkLow'] = addPriceRange(data['h'][::-1],data['l'][::-1],52)[1]
    data['rangescore'] = addPriceRangeScore(data['52wkHigh'],data['52wkLow'],data['c'])
    data['ema20'] = ema20.get('ema')
    data['ema200'] = ema200.get('ema')
    data['10dayHigh'] = addPriceRange(data['h'][::-1],data['l'][::-1],10)[0]
    data['10dayLow'] = addPriceRange(data['h'][::-1],data['l'][::-1],10)[1]
    data['spreadRatio'] = addMarketSpreadRatio(data['10dayHigh'],data['10dayLow'],52)
    data['relVol'] = addRelVol(data['v'],52)
    data['adx'] = adx.get('adx')
    stock.append(data)
    stock.append(i)
    result.append(stock)
  return result

# DOWNLOAD AND FORMAT DATA


In [ ]:
### Downloads a CSV File Containing Scraped Stock data
def assembleCSV(stock_data):
  drive.mount('/content/drive')
  path = '/content/drive/My Drive/output.csv'
  for i in stock_data:
    for y in range(len(i)-1):
      name = i[-1]
      time = i[y].get('t')
      open = i[y].get('o')
      high = i[y].get('h')
      low = i[y].get('l')
      close = i[y].get('c')
      volume = i[y].get('v')
      rsi = i[y].get('rsi')
      sma20 = i[y].get('sma20')
      sma200 = i[y].get('sma200')
      rate = i[y].get('cross')
      high52 = i[y].get('52wkHigh')
      low52 = i[y].get('52wkLow')
      rangescore = i[y].get('rangescore')
      ema20 = i[y].get('ema20')
      ema200 = i[y].get('ema200')
      high10 = i[y].get('10dayHigh')
      low10 = i[y].get('10dayLow')
      spreadRatio = i[y].get('spreadRatio')
      avVol = i[y].get('relVol')
      adx = i[y].get('adx')
      data = {'T':time[::-1],'O':open[::-1],'H':high[::-1],
              'L':low[::-1],'C':close[::-1],'V':volume[::-1],
              'RSI':rsi[::-1],'SMA20':sma20[::-1],'SMA200':sma200[::-1],
              'Rate':rate[::-1],'52wkHigh':high52[::-1],'52wkLow':low52[::-1],
              'RangeScore':rangescore[::-1],'EMA20':ema20[::-1],'EMA200':ema200[::-1],
              '10DayHigh':high10[::-1],'10DayLow':low10[::-1],
              'SpreadRatio':spreadRatio[::-1],'RelVol':avVol[::-1], 'ADX':adx[::-1]}
      df = pd.DataFrame(data)
      df = df.set_index('T')
      filename = name + 'stockData.csv'
      df.to_csv(filename)
      #files.download(filename)

# Indicator Analysis Functions

In [ ]:
### Shows the accuracy of crosses as indicators (death cross goes down, golden cross goes up)
#RESULT FORMAT : [DEATH CROSS ACCURACY PERCENTAGE , GOLDEN CROSS ACCURACY PERCENTAGE , DEATH CROSS DIFFERENCE PERCENTAGE , GOLD CROSS DIFFERENCE PERCENTAGE]
def check_cross_accuracy(length_of_comparison,tickers):
  final = []
  for name in tickers:
    death_cross = 0
    golden_cross = 0
    gold_accurate = 0
    death_accurate = 0
    death_difference_percentage = 0
    gold_difference_percentage = 0
    filename = name + 'stockData.csv'
    df = pd.read_csv(filename)
    for x in range(len(df.values)):
      if x-length_of_comparison >= 0:
        try:
          indicator = df.values[x][9]
          today_av = (df.values[x][1] + df.values[x][2] + df.values[x][3] + df.values[x][4]) /4
          tomorrow_av = (df.values[x-length_of_comparison][1] + df.values[x-length_of_comparison][2] + df.values[x-length_of_comparison][3] + df.values[x-length_of_comparison][4]) /4
          if indicator == 'golden cross':
            golden_cross += 1
            difference_percentage = (tomorrow_av - today_av) / today_av
            gold_difference_percentage += difference_percentage
            if today_av < tomorrow_av:
              gold_accurate += 1
          elif indicator == 'death cross':
            death_cross += 1
            difference_percentage = (tomorrow_av - today_av) / today_av
            death_difference_percentage += difference_percentage
            if today_av > tomorrow_av:
              death_accurate += 1
        except IndexError:
          continue
    death_score = death_accurate / death_cross
    gold_score = gold_accurate / golden_cross
    death_difference_percentage /= death_cross
    gold_difference_percentage /= golden_cross
    result = [death_score, gold_score, death_difference_percentage, gold_difference_percentage]
    final.append(result)
  return final

In [ ]:
### Shows the accuracy of RSI as an indicator (Oversold: price goes up, undersold: price goes down)
#RESULT FORMAT : [OVERSOLD ACCURACY PERCENTAGE , OVERBOUGHT ACCURACY PERCENTAGE , OVERSOLD AVERAGE DIFFERENCE PERCENTAGE , OVERBOUGHT AVERAGE DIFFERENCE PERCENTAGE]
def check_RSI(length_of_comparison,tickers):
  final = []
  for name in tickers:
    oversold = 0
    overbought = 0  
    sold_correct = 0
    bought_correct = 0
    bought_av_dif = 0
    sold_av_dif = 0
    filename = name + 'stockData.csv'
    df = pd.read_csv(filename)
    for x in range(len(df.values)):
      if x-length_of_comparison >= 0:
        try:
          indicator = df.values[x][6]
          today_av = (df.values[x][1] + df.values[x][2] + df.values[x][3] + df.values[x][4]) /4
          tomorrow_av = (df.values[x-length_of_comparison][1] + df.values[x-length_of_comparison][2] + df.values[x-length_of_comparison][3] + df.values[x-length_of_comparison][4]) /4
          if indicator > 60:
            overbought += 1
            difference_percent = (tomorrow_av - today_av)/today_av
            bought_av_dif += difference_percent
            if today_av > tomorrow_av:
              bought_correct += 1
          elif indicator < 30:
            oversold += 1
            difference_percent = (tomorrow_av - today_av) / today_av
            sold_av_dif += difference_percent
            if today_av < tomorrow_av:
              sold_correct += 1
        except IndexError:
          continue
    bought_av_dif /= overbought 
    sold_av_dif /= oversold
    bought_av_dif *= 100
    sold_av_dif *= 100
    sold_score = sold_correct / oversold 
    bought_score = bought_correct / overbought 
    result = [sold_score, bought_score, sold_av_dif, bought_av_dif]
    final.append(result)
  return final

In [ ]:
### Checks the accuracy of the Price Range Score as an indicator over a specified period of time 
### Score < 0.3 = oversold , score > o.7 = overbought
def checkPriceRangeScoreAccuracy(length_of_comparison,tickers):
  final = []
  for name in tickers:
    filename = name + 'stockData.csv'
    df = pd.read_csv(filename)
    high_correct = 0
    low_correct = 0 
    high_total = 0 
    low_total = 0
    for x in range(len(df.values)):
      if x-length_of_comparison >= 0:
        try:
          today_price = df.values[x][4]
          next_price = df.values[x - length_of_comparison][4]
          indicator = df.values[x][12]
          if indicator < 0.3:
            low_total += 1
            if today_price < next_price:
              low_correct += 1
          elif indicator > 0.7:
            high_total += 1
            if today_price > next_price:
              high_correct += 1
        except IndexError:
          continue
    low_ratio = low_correct / low_total 
    high_ratio = high_correct / high_total
    result = [low_ratio, high_ratio]
    final.append(result)
  return final

# ALGORITHM

In [ ]:
### Build algo here
### -50% stop loss
### One more indicator necessary
def algoStructure(length_of_comparison,tickers,cross_symbol,rsiScore,range_Score,adxScore):
  result = []
  for name in tickers:
    filename = name + 'stockData.csv'
    df = pd.read_csv(filename)
    trades = []
    for x in range(len(df.values)):
      rsi = df.values[x][6]
      rangeScore = df.values[x][12]
      spread_ratio = df.values[x][17]
      cross = df.values[x][9]
      adx = df.values[x][19]
      relVol = df.values[x][18]
      if x-length_of_comparison >= 0:
        #print(cross)
        if adx >= adxScore:
          if cross == cross_symbol:
            if rsi <= rsiScore:
              if rangeScore <= range_Score:
                try:
                  buy_price = df.values[x][4]
                  sell_price = df.values[x-length_of_comparison][4]
                  #STOP LOSS
                  for u in range(length_of_comparison+1):
                    dif = df.values[x-u][4] - buy_price
                    if dif <= (buy_price * -0.3):
                      sell_price = df.values[x-u][4]
                      #print('Stop Triggered at:',sell_price)
                  #
                  trade = [buy_price,sell_price,x]
                  trades.append(trade)
                except IndexError:
                  break
    if len(trades) > 0:
      result.append(trades)
  return result

In [ ]:
###Automatically decide how many shares of a stock to purchase based on select indicators
def adjustLeverage(leverage, adx, rsi, rangeScore, money, buyPrice):
  result = int(leverage * ((0.1 * money) / buyPrice))
  if adx >= 50:
    if rsi <= 25: 
      if rangeScore <= 0.2: 
        result = int(leverage * ((0.3 * money) / buyPrice)) 
  elif adx < 50:
    if rsi <= 30: 
      if rangeScore <= 0.3: 
        result = int(leverage * ((0.2 * money) / buyPrice))
  return result

In [ ]:
### Build algo here
### -50% stop loss
### Same as algo1, however this time it starts with $10,000 instead of infinite
def algo2(length_of_comparison,tickers,cross_symbol,rsiScore,range_Score,adxScore):
  result = []
  money = 20000
  for name in tickers:
    filename = name + 'stockData.csv'
    df = pd.read_csv(filename)
    trades = []
    for x in range(len(df.values)):
      rsi = df.values[x][6]
      rangeScore = df.values[x][12]
      spread_ratio = df.values[x][17]
      cross = df.values[x][9]
      adx = df.values[x][19]
      relVol = df.values[x][18]
      leverage = 1
      for u in range(len(trades)):
        if x == trades[u][2] + length_of_comparison:
          money += trades[u][1]
      if x-length_of_comparison >= 0:
        if adx >= adxScore:
          if cross == cross_symbol:
            if rsi <= rsiScore:
              if rangeScore <= range_Score:
                try:
                  buy_price = df.values[x][4]
                  sell_price = df.values[x-length_of_comparison][4]
                  leverage = adjustLeverage(leverage, adx, rsi, rangeScore, money, buy_price)
                  if money >= leverage * buy_price:
                    #STOP LOSS
                    for z in range(1,length_of_comparison):
                      dif = df.values[x-z][4] - buy_price
                      if dif <= (buy_price * -0.4):
                        sell_price = df.values[x-z][4]
                    #
                    trade = [leverage*buy_price,leverage*sell_price,x,rsi]
                    trades.append(trade)
                    money -= trade[0] 
                  elif money >= 1 * buy_price:
                    #STOP LOSS
                    for i in range(length_of_comparison):
                      dif = df.values[x-i][4] - buy_price
                      if dif <= (buy_price * -0.5):
                        sell_price = df.values[x-i][4]
                        #print('Stop Triggered at:',sell_price)
                    #
                    trade = [buy_price,sell_price,x,rsi]
                    trades.append(trade)
                    money -= trade[0] 
                except IndexError:
                  break
    if len(trades) > 0:
      result.append(trades)
  print(f'TOTAL MONEY: {money}')
  return result

In [ ]:
###Algorithm based purely on RSI
###Edit the data function to pull data by the minute
#### this algorithm seems to be successful but NEEDS TO RUN BY THE MINUTE

def algo3(length_of_comparison, tickers, rsiScore):
  result = []
  money = 20000
  for name in tickers:
    filename = name + 'stockData.csv'
    df = pd.read_csv(filename)
    trades = []
    for x in range(len(df.values)):
      rsi = df.values[x][6]
      leverage = 1
      for u in range(len(trades)):
        if x == trades[u][2] + length_of_comparison:
          money += trades[u][1]
      if x-length_of_comparison >= 0:
        if rsi <= rsiScore:
          try:
            buy_price = df.values[x][4]
            sell_price = df.values[x-length_of_comparison][4]
            if money >= leverage * buy_price:
              #STOP LOSS
              for z in range(1,length_of_comparison):
                dif = df.values[x-z][4] - buy_price
                if dif <= (buy_price * -0.4):
                  sell_price = df.values[x-z][4]
              #
              trade = [leverage*buy_price,leverage*sell_price,x,rsi]
              trades.append(trade)
              money -= trade[0] 
            elif money >= 1 * buy_price:
              #STOP LOSS
              for i in range(length_of_comparison):
                dif = df.values[x-i][4] - buy_price
                if dif <= (buy_price * -0.5):
                  sell_price = df.values[x-i][4]
                  #print('Stop Triggered at:',sell_price)
              #
              trade = [buy_price,sell_price,x,rsi]
              trades.append(trade)
              money -= trade[0] 
          except IndexError:
            break
    if len(trades) > 0:
      result.append(trades)
  print(f'TOTAL MONEY: {money}')
  return result

# LIVE ALGORITHM

In [ ]:
def send_message(message,url):
  webhook = Webhook.from_url(url, adapter=RequestsWebhookAdapter())
  webhook.send(message)

In [ ]:
###Undocumented
###return True if double peak detected, False otherwise
### Delta should be a negative number representing the difference ratio between the two peaks
def double_peak(prices,timeperiod,delta):
  #last price is the most recent
  peak1 = 0
  peak2 = 0
  for x in range(1, timeperiod):
    if prices[-x] > peak1:
      peak1 = prices[-x]
    elif prices[-x] > peak2 and prices[-x] < peak1:
      peak2 = prices[-x]
  difference = (peak2 - peak1) / peak1 
  if difference >= delta:
    return True
  return False
    ###HOW THE FUCK DO I DO THIS --> find two peaks, find the distance between these two peaks, use this  distance to determine the strength of the bearish trend

In [ ]:
### Gets most recent row of data from Finnhub
### UPDATE DOCUMENTATION
def getLatestRow(user, period, smoothingPeriod, largesmoothingPeriod):
  key= 'cc06epiad3idf21ispa0'
  client = finnhub.Client(api_key=key)
  tickers = user
  result = []
  secperday = 86400
  today = int(time.time())
  av300 = today - 25920000
  for i in tickers:
    stock = []
    data = client.technical_indicator(i,period,av300, today, indicator='rsi', indicator_fields={"timeperiod":smoothingPeriod})
    adx = client.technical_indicator(i,period, av300, today, indicator='adx', indicator_fields={"timeperiod":smoothingPeriod})
    sma20 = client.technical_indicator(i,period,av300, today, indicator='sma', indicator_fields={"timeperiod":smoothingPeriod})
    sma200 = client.technical_indicator(i,period,av300, today, indicator='sma', indicator_fields={"timeperiod":largesmoothingPeriod})
    data['sma20'] = sma20.get('sma')
    data['sma200'] = sma200.get('sma')
    data['52wkHigh'] = addPriceRange(data['h'][::-1],data['l'][::-1],52)[0]
    data['52wkLow'] = addPriceRange(data['h'][::-1],data['l'][::-1],52)[1]
    data['rangescore'] = addPriceRangeScore(data['52wkHigh'],data['52wkLow'],data['c'])
    data['10dayHigh'] = addPriceRange(data['h'][::-1],data['l'][::-1],10)[0]
    data['10dayLow'] = addPriceRange(data['h'][::-1],data['l'][::-1],10)[1]
    data['spreadRatio'] = addMarketSpreadRatio(data['10dayHigh'],data['10dayLow'],52)
    data['relVol'] = addRelVol(data['v'],52)
    data['adx'] = adx.get('adx')
    stock.append(data)
    stock.append(i)
    result.append(stock)
  return result

In [ ]:
### Gets most recent row of data from Finnhub
### UPDATE DOCUMENTATION
def getLatestRow1(user, period, smoothingPeriod, largesmoothingPeriod):
  key= 'cc06epiad3idf21ispa0'
  client = finnhub.Client(api_key=key)
  tickers = user
  result = []
  secperday = 86400
  today = int(time.time())
  av300 = today - 25920000
  for i in tickers:
    stock = []
    data = client.technical_indicator(i,period,av300, today, indicator='rsi', indicator_fields={"timeperiod":smoothingPeriod})
    data['52wkHigh'] = addPriceRange(data['h'][::-1],data['l'][::-1],52)[0]
    data['52wkLow'] = addPriceRange(data['h'][::-1],data['l'][::-1],52)[1]
    data['rangescore'] = addPriceRangeScore(data['52wkHigh'],data['52wkLow'],data['c'])
    data['10dayHigh'] = addPriceRange(data['h'][::-1],data['l'][::-1],10)[0]
    data['10dayLow'] = addPriceRange(data['h'][::-1],data['l'][::-1],10)[1]
    data['spreadRatio'] = addMarketSpreadRatio(data['10dayHigh'],data['10dayLow'],52)
    data['relVol'] = addRelVol(data['v'],52)
    stock.append(data)
    stock.append(i)
    result.append(stock)
  return result

In [ ]:
'''
Live version of the algorithm
Runs via discord notification
Sell security after 10 days of purchase
'''
def live_algo(tickers,rsiScore,adxScore,rangescore):
  url = 'https://discord.com/api/webhooks/1034561225863987220/6cUTwBisxvNmLyA9f_R8OmG44tm48-XFOi7ANeXBXMD6515CFDt7l0t69JoAIykHdT7K'
  run = True
  trades = []
  send_message('BOT IS NOW LIVE', url)
  while run:
    try:
      data = getLatestRow(tickers, 'D', 20, 100)
      for x in range(len(data)):
        name = data[x][1]
        o = data[x][0].get('o')[-1]
        h = data[x][0].get('h')[-1]
        l = data[x][0].get('l')[-1]
        c = data[x][0].get('c')[-1]
        rsi = data[x][0].get('rsi')[-1]
        adx = data[x][0].get('adx')[-1]
        sma20 = data[x][0].get('sma20')[-1]
        sma200 = data[x][0].get('sma200')[-1]
        rangeScore = data[x][0].get('rangescore')[-1]
        if sma20 > sma200:
          if adx >= adxScore:
            if rsi <= rsiScore:
              if rangeScore <= rangescore:
                send_message(f'Buy {name}. RSI: {rsi}. ADX: {adx}. STOCHASTIC: {rangeScore}.', url)
                stock = ['B',name, c]
                trades.append(stock)
      time.sleep(120)
    except KeyboardInterrupt:
      send_message('BOT IS NOW OFF', url)
      return trades


#tickers = ['AAPL','BAC','KO','CVX','AXP','SPY']
tickers = ['SOFI']
#live_algo(tickers,40,20,0.29)

In [ ]:
### UNCODOCUMENTED
'''
Live daytrading algorithm
Runs off the discord server
'''
def liveDayTrader(tickers, rsiScore, adxScore, rangescore, sell, stop):
  run = True
  trades = []
  url = 'https://discord.com/api/webhooks/1035201248703557734/JdwtCCc-VwosPn6kZ8H2VvcLKF23PRcK-9AZM-daIm3Bshgoo3yUT8lc4fR8O0EZYdNB'
  send_message('BOT IS NOW LIVE', url)
  tradeID = 1
  while run:
    try:
      ### This call will pull prices by the minute
      data = getLatestRow(tickers, '1', 7, 21)
      for x in range(len(data)):
        name = data[x][1]
        t = data[x][0].get('t')[-1]
        o = data[x][0].get('o')[-1]
        h = data[x][0].get('h')[-1]
        l = data[x][0].get('l')[-1]
        c = data[x][0].get('c')[-1]
        rsi = data[x][0].get('rsi')[-1]
        adx = data[x][0].get('adx')[-1]
        sma20 = data[x][0].get('sma20')[-1]
        sma200 = data[x][0].get('sma200')[-1]
        rangeScore = data[x][0].get('rangescore')[-1]
        ###Check for double peak (bearish)
        if double_peak(data[x][0].get('c'), 60, 0.1):
          send_message('{name} DOUBLE PEAK', url)
        ###
        if rsi < rsiScore:
          if sma20 > sma200:
            stock = ['B', name, c, t, tradeID]
            send_message(f'Buy {name} at {c}.           ID: {tradeID}', url)
            trades.append(stock)
            tradeID += 1
        ### SEll SIGNAL
        if len(trades) > 0:
          for y in trades:
            if y[0] == 'B':
              if name == y[1]:
                last_c = y[2]
                change = (c - last_c) / last_c
                if change >= sell:
                  send_message(f'Sell {name} at {c}.           ID: {y[4]}', url)
                  stock = ['S', name, c, t, y[4]]
                  trades.append(stock)
        ###
                 ### STOP LOSS
                elif change <= stop:
                  send_message(f'[STOP LOSS] Sell {name} at {c}           ID: {y[4]}', url)
                  stock = ['SL', name, c, t, y[4]] 
                  trades.append(stock)
                ###
      time.sleep(90)
    except KeyboardInterrupt:
      send_message('BOT IS NOW OFF', url)
      return trades

tickers = ['SPY']
#liveDayTrader(tickers, 38, 20, 0.29, 0.01, -0.5)

#RSI ALGO

In [ ]:
###Live Discord Based Algorithm

def rsi_ALGO(tickers,rsi):
  run = True
  trades = []
  url = 'https://discord.com/api/webhooks/1044794416080093184/SJLr28athMuTwB2b8iv7_cLyqEP4KoiB3fBFHwEzy0NDGBN2DKnkdFpmwRbXIfdT8HYV'
  send_message('BOT IS NOW LIVE', url)
  tradeID = 1
  while run:
    try:
      ### This call will pull prices by the minute
      data = getLatestRow(tickers, '1', 7, 21)
      for x in range(len(data)):
        name = data[x][1]
        t = data[x][0].get('t')[-1]
        c = data[x][0].get('c')[-1]
        rsi = data[x][0].get('rsi')[-1]
        if rsi <= rsi:
          send_message(f"Buy {name} at {c}.",url)
          stock = ['B', name, c, t, tradeID, rsi]
          trades.append(stock)
          tradeID += 1
        ###Sell Criteria
        if len(trades) > 0:
          for trade in trades:
            if trade[0] == 'B':
              if name == trade[1]:
                difference = (c - trade[2]) / trade[2]
                if difference >= 0.01:
                  send_message(f"Sell {name} at {c}.",url)
                  stock = ['S', name, c, t, tradeID]
                  trades.append(stock)
      time.sleep(90)
    except KeyboardInterrupt:
      send_message('BOT IS NOW OFF', url)
      return trades

tickers = ['SPY']
#rsi_ALGO(tickers, 38)

# Algorithm Analysis Functions

In [ ]:
### Under development
def getBadTrades(trades,tickers):
  result = []
  for item in range(len(trades)):
    bad_trades = []
    for trade in trades[item]:
      ticker = tickers[item]
      buy = trade[0]
      sell = trade[1]
      profit = sell - buy
      index = trade[2] 
      if profit < 0:
        badTrade = [ticker,buy,sell,profit,index]
        bad_trades.append(badTrade)
    result.append(bad_trades)
  return result

In [ ]:
### UNDER DEVELOPMENT
def getGoodTrades(trades,tickers):
  result = []
  for item in range(len(trades)):
    good_trades = []
    for trade in trades[item]:
      ticker = tickers[item]
      buy = trade[0]
      sell = trade[1]
      profit = sell - buy
      index = trade[2] 
      if profit > 0:
        goodTrade = [ticker,buy,sell,profit,index]
        good_trades.append(goodTrade)
    result.append(good_trades)
  return result

In [ ]:
def getStandardDeviation(df,buyIndex,u,trades,indicator):
  sum = 0
  for trade in df:
    sum += (df[buyIndex][indicator] - u) ** 2
  sum /= len(df)
  result = math.sqrt(sum)
  return result

In [ ]:
### find the mean and standard deviation of a certain indicator 
def analyzeTrades(trades,tickers, indicator):
  '''
  ARGS: 
    indicator (string): indicator user wants to study. rsi, rangescore, or spreadratio.
  '''
  result = []
  indicators = {'rsi': 6, 'rangescore': 12, 'spreadratio': 17}
  for ticker in range(len(trades)):
    filename = tickers[ticker] + 'stockData.csv'
    df = pd.read_csv(filename)
    if len(trades[ticker]) > 0:
      mean = 0
      n = len(trades[ticker])
      for y in trades[ticker]:
        buyIndex = y[4] 
        sellIndex = y[4] - 10
        mean += df.values[buyIndex][indicators.get(indicator)]
      mean /= n
      std = getStandardDeviation(df.values, buyIndex, mean, trades, indicators.get(indicator))
      data = [mean, std]
      result.append(data)
  return result

In [ ]:
def check_algo(trades,tickers):
  total = 0
  av_rate_of_success = 0
  denominator = 0
  totalTrades = 0
  for trade in range(len(trades)):
    if len(trades[trade]) > 0:
      profit = 0
      denominator += 1
      good = 0
      total_trades = len(trades[trade])
      for x in trades[trade]:
        buy = x[0]
        sell = x[1]
        index = x[2]
        difference = sell - buy
        profit += difference
        total += difference 
        if difference >= 0:
          good += 1 
      ratio = good / total_trades
      av_rate_of_success += ratio 
      totalTrades += total_trades
  if denominator > 0:
    av_rate_of_success /= denominator
    return [total, av_rate_of_success, totalTrades]
  else:
    return [0,0]

In [ ]:
### Returns most accurate numbers 
def analyzeAlgo(tickers):
  result = []
  cross_symbol = 'golden cross'
  highProfit = 0
  highRate = 0
  adxScore = 20
  for x in range(30):
    timeperiod = x +1
    for i in range(10,40):
      rsiScore = i + 1
      range_score = 0
      for u in range(40):
        range_score += 0.01
        trades = algoStructure(timeperiod,tickers,cross_symbol,rsiScore,range_score,adxScore)
        if len(trades) > 0:
          data = check_algo(trades,tickers)
          profit = data[0]
          successRate = data[1]
          totalTrades = data[2]
          if totalTrades >= 10 :
            if profit > highProfit:
                highProfit = profit
                if successRate >= (highRate - 0.1):
                  highRate = successRate
                  rsi = rsiScore
                  rangeScore = range_score
                  timePeriod = timeperiod
  result = [rsi,rangeScore,timePeriod]
  return result


# RUN

In [ ]:
 ####### RUN OBSERVATIONS IN HERE ########
###BELOW TICKERS ARE TAKEN FROM BERKSHIRE HATHAWAY HOLDINGS
berkshireTickers = ['AAPL','BAC','KO','CVX','AXP','SPY']
pelosiTickers = ['AAPL', 'MSFT', 'FB', 'AMZN']
def get_csv(tickers):
  data = getFinnhubData(tickers)
  assembleCSV(data)

def main(tickers):
  get_csv(tickers)
  trades = algo2(10,tickers,'golden cross',40,0.29,20)
  trades1 = algo3(1,tickers,40)
main(berkshireTickers)

1.   Bollinger Bands 
2.   On-Balance Volume 
>>>Make a function that determines how many shares of a security to purchase based on indicators 


